## Epidemiological Simulation Project Using the SEIRD Model

In [11]:
# Imports
from scipy.integrate import odeint
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
import os
import plotly.io as pio
from ipywidgets import IntSlider, FloatSlider, Layout, Dropdown, interact_manual, VBox, HBox
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Layout

# Plotly renderer configuration
pio.renderers.default = "plotly_mimetype+notebook"

# Functions
def ode_model(z, t, beta, sigma, gamma, mu):
    S, E, I, R, D = z
    N = S + E + I + R + D
    dSdt = -beta*S*I/N
    dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I - mu*I
    dRdt = gamma*I
    dDdt = mu*I
    return [dSdt, dEdt, dIdt, dRdt, dDdt]

def ode_solver(t, initial_conditions, params):
    initE, initI, initR, initN, initD = initial_conditions
    beta, sigma, gamma, mu = params
    initS = initN - (initE + initI + initR + initD)
    res = odeint(ode_model, [initS, initE, initI, initR, initD], t, args=(beta, sigma, gamma, mu))
    return res

def main(initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days):
    initial_conditions = [initE, initI, initR, initN, initD]
    params = [beta, sigma, gamma, mu]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, I, R, D = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Recovered'))
    fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines+markers',name='Death'))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.update_layout(title='Simulation of SEIRD Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                       width=700, height=400,
                       margin=dict(l=50, r=50, t=60, b=50)
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.show()


### Running the simulation

In [ ]:


# Default values for sliders
initE = 10
initI = 5
initR = 0
initD = 0
initN = 100000
beta = 0.5
sigma = 0.2
gamma = 0.1
mu = 0.01
days = 180

# Simple style and layout for sliders
style = {'description_width': '120px'}
slider_layout = Layout(width='400px')

interact(main, 
         initE=IntSlider(min=0, max=100000, step=1, value=initE, description='Initial Exposed', style=style, layout=slider_layout),
         initI=IntSlider(min=0, max=100000, step=10, value=initI, description='Initial Infected', style=style, layout=slider_layout),
         initR=IntSlider(min=0, max=100000, step=10, value=initR, description='Initial Recovered', style=style, layout=slider_layout),
         initD=IntSlider(min=0, max=100000, step=10, value=initD, description='Initial Deaths', style=style, layout=slider_layout),
         initN=IntSlider(min=0, max=1380000000, step=1000, value=initN, description='Initial Population', style=style, layout=slider_layout),
         beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
         sigma=FloatSlider(min=0, max=4, step=0.01, value=sigma, description='Incubation rate', style=style, layout=slider_layout),
         gamma=FloatSlider(min=0, max=4, step=0.01, value=gamma, description='Recovery rate', style=style, layout=slider_layout),
         mu=FloatSlider(min=0, max=1, step=0.01, value=mu, description='Mortality rate', style=style, layout=slider_layout),
         days=IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout)
        );

interactive(children=(IntSlider(value=10, description='Initial Exposed', layout=Layout(width='400px'), max=100…

In [ ]:
# Presets de doenças
disease_presets = {
    'Custom': {'beta': 0.5, 'sigma': 0.2, 'gamma': 0.1, 'mu': 0.01},
    'COVID-19 (Original)': {'beta': 0.5, 'sigma': 0.196, 'gamma': 0.1, 'mu': 0.02},
    'COVID-19 (Delta)': {'beta': 0.8, 'sigma': 0.196, 'gamma': 0.1, 'mu': 0.015},
    'COVID-19 (Omicron)': {'beta': 1.2, 'sigma': 0.33, 'gamma': 0.143, 'mu': 0.005},
    'Influenza': {'beta': 0.7, 'sigma': 0.5, 'gamma': 0.143, 'mu': 0.001},
    'Measles': {'beta': 1.8, 'sigma': 0.125, 'gamma': 0.143, 'mu': 0.002}
}

# Default values for sliders
initE = 10
initI = 5
initR = 0
initD = 0
initN = 100000
days = 180

# Simple style and layout for sliders
style = {'description_width': '120px'}
slider_layout = Layout(width='400px')

# Criar dropdown para seleção de doença
disease_dropdown = Dropdown(
    options=list(disease_presets.keys()),
    value='COVID-19 (Original)',
    description='Disease:',
    style=style,
    layout=slider_layout
)

def create_interactive_simulation():
    # Sliders que sempre podem ser ajustados
    initE_slider = IntSlider(min=0, max=100000, step=1, value=initE, description='Initial Exposed', style=style, layout=slider_layout)
    initI_slider = IntSlider(min=0, max=100000, step=10, value=initI, description='Initial Infected', style=style, layout=slider_layout)
    initR_slider = IntSlider(min=0, max=100000, step=10, value=initR, description='Initial Recovered', style=style, layout=slider_layout)
    initD_slider = IntSlider(min=0, max=100000, step=10, value=initD, description='Initial Deaths', style=style, layout=slider_layout)
    initN_slider = IntSlider(min=0, max=1380000000, step=1000, value=initN, description='Initial Population', style=style, layout=slider_layout)
    days_slider = IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout)
    
    # Sliders de parâmetros epidemiológicos (serão atualizados baseado na doença)
    beta_slider = FloatSlider(min=0, max=4, step=0.01, value=0.5, description='Infection rate (β)', style=style, layout=slider_layout)
    sigma_slider = FloatSlider(min=0, max=2, step=0.001, value=0.196, description='Incubation rate (σ)', style=style, layout=slider_layout)
    gamma_slider = FloatSlider(min=0, max=1, step=0.001, value=0.1, description='Recovery rate (γ)', style=style, layout=slider_layout)
    mu_slider = FloatSlider(min=0, max=0.1, step=0.001, value=0.02, description='Mortality rate (μ)', style=style, layout=slider_layout)
    
    def update_disease_params(change):
        if change['new'] != 'Custom':
            preset = disease_presets[change['new']]
            beta_slider.value = preset['beta']
            sigma_slider.value = preset['sigma']
            gamma_slider.value = preset['gamma']
            mu_slider.value = preset['mu']
    
    disease_dropdown.observe(update_disease_params, names='value')
    
    # Função que será chamada quando os parâmetros mudarem
    def run_simulation(disease, initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days):
        main(initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days)
    
    # Interface interativa
    interact_manual(run_simulation,
                   disease=disease_dropdown,
                   initE=initE_slider,
                   initI=initI_slider, 
                   initR=initR_slider,
                   initD=initD_slider,
                   initN=initN_slider,
                   beta=beta_slider,
                   sigma=sigma_slider,
                   gamma=gamma_slider,
                   mu=mu_slider,
                   days=days_slider)


create_interactive_simulation()

interactive(children=(Dropdown(description='Disease:', index=1, layout=Layout(width='400px'), options=('Custom…

In [15]:
def add_pie_chart(S, E, I, R, D, days):
    # Valores finais
    final_values = [S[-1], E[-1], I[-1], R[-1], D[-1]]
    labels = ['Susceptible', 'Exposed', 'Infected', 'Recovered', 'Deaths']
    colors = ['lightblue', 'orange', 'red', 'green', 'black']
    
    fig_pie = go.Figure(data=[go.Pie(labels=labels, values=final_values, 
                                     marker_colors=colors)])
    fig_pie.update_layout(title=f'Population Distribution at Day {days}',
                         width=400, height=400)
    return fig_pie
def calculate_metrics(S, E, I, R, D, beta, gamma, mu, initN):
    # R0 (Número básico de reprodução)
    R0 = beta / (gamma + mu)
    
    # Pico de infectados
    peak_infected = max(I)
    peak_day = np.argmax(I)
    
    # Taxa de ataque final
    attack_rate = (initN - S[-1]) / initN * 100
    
    # Mortalidade final
    mortality_rate = D[-1] / initN * 100
    
    return {
        'R0': R0,
        'Peak Infected': int(peak_infected),
        'Peak Day': int(peak_day),
        'Attack Rate (%)': round(attack_rate, 2),
        'Mortality Rate (%)': round(mortality_rate, 4)
    }
def plot_daily_incidence(sol, tspan):
    S, E, I, R, D = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4]
    
    # Calcular novos casos diários
    new_infections = np.diff(np.concatenate([[0], initN - S]))
    new_deaths = np.diff(np.concatenate([[0], D]))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan[:-1], y=new_infections, 
                            mode='lines', name='New Infections'))
    fig.add_trace(go.Scatter(x=tspan[:-1], y=new_deaths, 
                            mode='lines', name='New Deaths'))
    
    fig.update_layout(title='Daily Incidence',
                     xaxis_title='Day',
                     yaxis_title='Daily Cases',
                     width=700, height=300)
    return fig
def main(initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days):
    initial_conditions = [initE, initI, initR, initN, initD]
    params = [beta, sigma, gamma, mu]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, I, R, D = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4]
    
    # Gráfico principal
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines', name='Recovered'))
    fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines', name='Deaths'))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    fig.update_layout(title='SEIRD Model Simulation',
                     xaxis_title='Day',
                     yaxis_title='Population',
                     width=800, height=400)
    fig.update_xaxes(tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.show()
    
    # Métricas importantes
    metrics = calculate_metrics(S, E, I, R, D, beta, gamma, mu, initN)
    print("\n📊 Key Metrics:")
    for key, value in metrics.items():
        print(f"• {key}: {value}")
    
    # Gráfico de pizza (distribuição final)
    fig_pie = add_pie_chart(S, E, I, R, D, days)
    fig_pie.show()
    
    # Incidência diária
    if days > 7:  # Só mostrar se tiver dados suficientes
        fig_inc = plot_daily_incidence(sol, tspan)
        fig_inc.show()
main(initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days)



📊 Key Metrics:
• R0: 4.545454545454546
• Peak Infected: 27679
• Peak Day: 62
• Attack Rate (%): 98.88
• Mortality Rate (%): 8.9893
